# Preliminary analysis: What is in the dataset?

## Importing all the dependencies

In [22]:
import pandas as pd
%matplotlib inline

## Reading the data

In [23]:
data = pd.read_csv("data/Evaluaciones_Agropecuarias_Municipales_EVA.csv")

In [24]:
data.head()

,CÓD. \nDEP.,DEPARTAMENTO,CÓD. MUN.,MUNICIPIO,GRUPO \nDE CULTIVO,SUBGRUPO \nDE CULTIVO,CULTIVO,DESAGREGACIÓN REGIONAL Y/O SISTEMA PRODUCTIVO,AÑO,PERIODO,Área Sembrada\n(ha),Área Cosechada\n(ha),Producción\n(t),Rendimiento\n(t/ha),ESTADO FISICO PRODUCCION,NOMBRE \nCIENTIFICO,CICLO DE CULTIVO
0,15,BOYACA,15114,BUSBANZA,HORTALIZAS,ACELGA,ACELGA,ACELGA,2006,2006B,2,1,1,1.00,FRUTO FRESCO,BETA VULGARIS,TRANSITORIO
1,25,CUNDINAMARCA,25754,SOACHA,HORTALIZAS,ACELGA,ACELGA,ACELGA,2006,2006B,82,80,1440,18.00,FRUTO FRESCO,BETA VULGARIS,TRANSITORIO
2,25,CUNDINAMARCA,25214,COTA,HORTALIZAS,ACELGA,ACELGA,ACELGA,2006,2006B,2,2,26,17.33,FRUTO FRESCO,BETA VULGARIS,TRANSITORIO
3,54,NORTE DE SANTANDER,54405,LOS PATIOS,HORTALIZAS,ACELGA,ACELGA,ACELGA,2006,2006B,3,3,48,16.00,FRUTO FRESCO,BETA VULGARIS,TRANSITORIO
4,54,NORTE DE SANTANDER,54518,PAMPLONA,HORTALIZAS,ACELGA,ACELGA,ACELGA,2006,2006B,1,1,5,10.00,FRUTO FRESCO,BETA VULGARIS,TRANSITORIO


Las variables importantes son: Municipio, Cod Municipio, Grupo de Cultivo, Subgrupo de cultivo, Cultivo, Desagregación regional y/o sistema productivo, año, periodo, rendimiento, ciclo de cultivo

#### Rendimiento se da en toneladas producidas por hectarea cosechada, es decir, entre más rendimiento significa que se sacan más toneladas en un area más pequeña. No tiene nada que ver con el precio, sería bueno conocer el precio del producto

## Changing the column names

In [25]:
data.columns = ['COD_DEP', 'DEPARTAMENTO', 'COD_MUN','MUNICIPIO','GRUP_CULTIVO','SUBGRUP_CULTIVO','CULTIVO','DESAGREGACION_SIST_PRODUCTIVO','YEAR','PERIODO','AREA_SEMBR','AREA_COSECH','PRODUCCION','RENDIMIENTO','ESTADO_PRODUCCION','NOMBRE_CIENTI','CICLO_CULTIVO']

In [26]:
data.head()

,COD_DEP,DEPARTAMENTO,COD_MUN,MUNICIPIO,GRUP_CULTIVO,SUBGRUP_CULTIVO,CULTIVO,DESAGREGACION_SIST_PRODUCTIVO,YEAR,PERIODO,AREA_SEMBR,AREA_COSECH,PRODUCCION,RENDIMIENTO,ESTADO_PRODUCCION,NOMBRE_CIENTI,CICLO_CULTIVO
0,15,BOYACA,15114,BUSBANZA,HORTALIZAS,ACELGA,ACELGA,ACELGA,2006,2006B,2,1,1,1.00,FRUTO FRESCO,BETA VULGARIS,TRANSITORIO
1,25,CUNDINAMARCA,25754,SOACHA,HORTALIZAS,ACELGA,ACELGA,ACELGA,2006,2006B,82,80,1440,18.00,FRUTO FRESCO,BETA VULGARIS,TRANSITORIO
2,25,CUNDINAMARCA,25214,COTA,HORTALIZAS,ACELGA,ACELGA,ACELGA,2006,2006B,2,2,26,17.33,FRUTO FRESCO,BETA VULGARIS,TRANSITORIO
3,54,NORTE DE SANTANDER,54405,LOS PATIOS,HORTALIZAS,ACELGA,ACELGA,ACELGA,2006,2006B,3,3,48,16.00,FRUTO FRESCO,BETA VULGARIS,TRANSITORIO
4,54,NORTE DE SANTANDER,54518,PAMPLONA,HORTALIZAS,ACELGA,ACELGA,ACELGA,2006,2006B,1,1,5,10.00,FRUTO FRESCO,BETA VULGARIS,TRANSITORIO


## Describing the data

In [27]:
data.describe()

,COD_DEP,COD_MUN,YEAR,AREA_SEMBR,AREA_COSECH,PRODUCCION,RENDIMIENTO
count,206068.000000,206068.000000,206068.000000,206068.000000,206068.000000,2.060680e+05,202635.000000
mean,40.322563,40747.534066,2012.455976,291.073825,249.419522,2.790726e+03,9.238820
std,25.278662,25256.621906,3.479113,1153.602556,980.375226,4.511471e+04,14.888659
min,5.000000,5001.000000,2006.000000,0.000000,0.000000,0.000000e+00,0.030000
25%,17.000000,17524.000000,2009.000000,10.000000,8.000000,3.200000e+01,1.500000
50%,41.000000,41357.000000,2013.000000,35.000000,30.000000,1.400000e+02,5.000000
75%,68.000000,68001.000000,2015.000000,151.000000,130.000000,6.480000e+02,11.230000
max,99.000000,99773.000000,2018.000000,47403.000000,38600.000000,4.546116e+06,246.000000


#### 

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206068 entries, 0 to 206067
Data columns (total 17 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   COD_DEP                        206068 non-null  int64  
 1   DEPARTAMENTO                   206068 non-null  object 
 2   COD_MUN                        206068 non-null  int64  
 3   MUNICIPIO                      206067 non-null  object 
 4   GRUP_CULTIVO                   206068 non-null  object 
 5   SUBGRUP_CULTIVO                206068 non-null  object 
 6   CULTIVO                        206068 non-null  object 
 7   DESAGREGACION_SIST_PRODUCTIVO  206068 non-null  object 
 8   YEAR                           206068 non-null  int64  
 9   PERIODO                        206068 non-null  object 
 10  AREA_SEMBR                     206068 non-null  int64  
 11  AREA_COSECH                    206068 non-null  int64  
 12  PRODUCCION                    

## Rendimiento de cultivos por departamento:

In [29]:
rendimiento_dep = data.groupby('DEPARTAMENTO').RENDIMIENTO.agg(['count','min','max','mean'])
rendimiento_dep.sort_values('mean',ascending=False).head(10)

,count,min,max,mean
DEPARTAMENTO,,,,
RISARALDA,2761,0.10,158.43,14.872112
VALLE DEL CAUCA,15633,0.05,200.69,13.961407
CALDAS,5273,0.14,156.07,11.795316
QUINDIO,3818,0.15,180.00,11.588499
ANTIOQUIA,18165,0.03,240.00,11.403534
NORTE DE SANTANDER,9638,0.11,150.00,10.794193
META,5059,0.20,100.00,10.478982
CUNDINAMARCA,17554,0.06,210.00,10.319582
BOYACA,20383,0.07,200.00,9.929606


## Rendimiento de cultivo

In [30]:
rendimiento_cult = data.groupby('CULTIVO').RENDIMIENTO.agg(['count','min','max','mean'])
rendimiento_cult.sort_values('mean',ascending=False).head(10)

,count,min,max,mean
CULTIVO,,,,
CAÑA AZUCARERA,619,48.17,200.69,117.301389
TOMATE,9641,0.50,246.00,38.291158
CHAMPIÑON,11,2.00,60.00,34.181818
PIÑA,2125,0.24,184.00,28.946936
SABILA,238,1.00,100.00,28.369622
MAIZ FORRAJERO,234,1.42,87.27,28.225983
CHAMBA,24,0.50,50.00,27.537500
CEBOLLIN,14,6.00,50.00,25.642857
CORDELINE CINTA,4,25.00,25.00,25.000000


## How many crops types are there?

In [31]:
rendimiento_cult.count()

count    223
min      220
max      220
mean     220
dtype: int64

## Relevant questions

1. Cuales son los cultivos más propicios a dar un buen rendimiento?
2. Cuales son los departamentos y municipios donde hay mayor rendimiento?
3. Con esto se podrían definir X cultivos que pueden ser aceptados de acuerdo con su rendimiento y el municipio donde se presenta el proyecto

## Example: ¿cuales son los 10 cultivos más propicios para cultivar en Marinilla-Antioquia?

In [32]:
rendimiento_marinilla = data[data.MUNICIPIO == "MARINILLA"].groupby('CULTIVO').RENDIMIENTO.agg(['count','min','max','mean'])
rendimiento_marinilla.sort_values('mean',ascending=False).head(10)

,count,min,max,mean
CULTIVO,,,,
REPOLLO,24,68.0,70.90,69.870833
CALABACIN,5,60.0,61.20,60.418000
TOMATE,24,20.0,86.82,49.369167
ZANAHORIA,24,26.0,30.24,29.593333
LECHUGA,24,27.0,30.00,28.042500
ACELGA,5,27.0,28.10,27.356000
COLIFLOR,24,24.0,27.00,25.109583
BROCOLI,24,20.0,27.00,23.642917
APIO,24,21.0,23.00,22.476250


# Relevant question 1: What are the crops that give the highest yield?

## For Colombia

In [33]:
stat = data.groupby(data['CULTIVO']).mean()
highest_crops = stat.nlargest(10, 'RENDIMIENTO').reset_index()
highest_crops

,CULTIVO,COD_DEP,COD_MUN,YEAR,AREA_SEMBR,AREA_COSECH,PRODUCCION,RENDIMIENTO
0,CAÑA AZUCARERA,58.664013,59092.257962,2012.519108,4492.603503,3694.469745,445074.662420,117.301389
1,TOMATE,40.042469,40471.413715,2012.285788,21.316035,19.770561,749.782992,38.291158
2,CHAMPIÑON,47.363636,47506.000000,2016.090909,1.181818,1.181818,50.000000,34.181818
3,PIÑA,49.255646,49656.476965,2013.026649,114.385727,81.794038,3352.271003,28.946936
4,SABILA,54.169014,54559.443662,2014.855634,14.482394,10.915493,262.686620,28.369622
5,MAIZ FORRAJERO,43.606838,43986.978632,2014.871795,103.145299,100.196581,3240.688034,28.225983
6,CHAMBA,16.541667,16824.208333,2012.333333,4.958333,4.541667,121.166667,27.537500
7,CEBOLLIN,10.933333,11567.266667,2015.000000,8.333333,8.066667,324.000000,25.642857
8,CORDELINE CINTA,63.000000,63065.500000,2011.500000,1.250000,1.250000,30.750000,25.000000
9,FRESA,31.855319,32331.154610,2013.607092,29.618440,23.974468,887.296454,24.435326


In [34]:
import plotly.express as px
fig = px.bar(highest_crops, x='CULTIVO', y='RENDIMIENTO',color='RENDIMIENTO')
fig.show()

## Yields by department

In [14]:
rendimiento_dep = data.groupby('DEPARTAMENTO').mean()
print("there are",len(rendimiento_dep),"departments")
highest_department = rendimiento_dep.nlargest(32, 'RENDIMIENTO').reset_index()
highest_department


there are 32 departments


,DEPARTAMENTO,COD_DEP,COD_MUN,YEAR,AREA_SEMBR,AREA_COSECH,PRODUCCION,RENDIMIENTO
0,RISARALDA,66.0,66313.425267,2012.486121,384.056940,328.105338,2728.612456,14.872112
1,VALLE DEL CAUCA,76.0,76433.679790,2012.225117,271.558451,233.968302,14701.273425,13.961407
2,CALDAS,17.0,17435.290341,2012.502808,320.766005,262.973044,2011.553538,11.795316
3,QUINDIO,63.0,63332.061590,2011.828776,210.800409,183.170713,1606.499872,11.588499
4,ANTIOQUIA,5.0,5425.417613,2012.645930,278.390373,241.832400,1963.080868,11.403534
5,NORTE DE SANTANDER,54.0,54401.391242,2012.380884,201.049841,186.230951,1098.033638,10.794193
6,META,50.0,50384.654264,2012.577562,935.364625,756.780195,5453.904598,10.478982
7,CUNDINAMARCA,25.0,25481.192699,2012.558102,187.811177,171.304409,1683.105588,10.319582
8,BOYACA,15.0,15486.188812,2012.314444,94.105317,81.186674,959.788783,9.929606
9,ARAUCA,81.0,81413.710618,2012.724621,1154.988331,1036.718786,8545.149358,9.345602


In [15]:
fig = px.bar(highest_department, x='DEPARTAMENTO', y='RENDIMIENTO',color='RENDIMIENTO')
fig.show()

# Relevant question 2: What are the municipios that have the highest yield

## In general

In [16]:
rendimiento_mun = data.groupby('MUNICIPIO').RENDIMIENTO.mean()
rendimiento_mun=rendimiento_mun.sort_values(ascending=False).head(10).to_frame().reset_index()
rendimiento_mun

,MUNICIPIO,RENDIMIENTO
0,SACHICA,55.339432
1,CAROLINA,33.268281
2,LA PINTADA,31.500000
3,PEÑOL,31.283008
4,ENTRERRIOS,31.237416
5,ENVIGADO,30.352593
6,GUATAPE,28.519118
7,SORA,28.080833
8,JARDIN,28.017064
9,DON MATIAS,27.102410


In [17]:
fig = px.bar(rendimiento_mun, x='MUNICIPIO', y='RENDIMIENTO',color='RENDIMIENTO')
fig.show()

# Relevant question 3: How is the data for a sample?

With this we can just give the algorithm a municipal or department and it will give us:
1. Top 5 Crops
2. Is the yield increasing or decreasing over time
3. Is the sample in the top 5 of country? if its a municipal its in the top 5 of the departament?

In [18]:
sample = "MARINILLA"
#sample = "Antioquia"
#sample = "Valle del cauca"

## What are the top 5 crops?

In [19]:
rendimiento_marinilla = data[data.MUNICIPIO == sample].groupby('CULTIVO').RENDIMIENTO.mean()
rendimiento_marinilla=rendimiento_marinilla.sort_values(ascending=False).head(5).to_frame().reset_index()
rendimiento_marinilla
#rendimiento_marinilla.sort_values('mean',ascending=False).head(10)

,CULTIVO,RENDIMIENTO
0,REPOLLO,69.870833
1,CALABACIN,60.418000
2,TOMATE,49.369167
3,ZANAHORIA,29.593333
4,LECHUGA,28.042500


In [20]:
fig = px.bar(rendimiento_marinilla, x='CULTIVO', y='RENDIMIENTO',color='RENDIMIENTO')
fig.show()

## How is the yield over time? is it increasing? decreasing?

In [21]:
rendimiento_marinilla = data[data.MUNICIPIO == sample].groupby('CULTIVO').RENDIM


AttributeError: 'DataFrameGroupBy' object has no attribute 'RENDIM'

## Is the sample in the top 5 of country? if it's a municipality, is it in the top 5 of the departament?